Moteur du courant EAC apportant les larves pour la renaissance des coraux

In [95]:
import matplotlib
matplotlib.use('TkAgg') 
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.animation import FuncAnimation
import cartopy.crs as ccrs 
import cartopy.feature as cfeature 
import numpy as np

# --- 1. GÉOMÉTRIE PRÉCISE DE LA CÔTE (Guidage) ---
def get_reef_curve(lat):
    """
    Retourne la longitude approximative du récif pour une latitude donnée.
    Permet de dessiner une GBR courbe et pas droite.
    """
    # Polynôme ajusté pour coller à la forme Queensland/PNG
    # À -8 (PNG) -> 144.0
    # À -14 (Cape York) -> 144.5
    # À -23 (Rockhampton) -> 152.0
    return 144.5 + 0.02 * (np.abs(lat) - 8)**2.2

# --- 2. CRÉATION DU RÉCIF (SUBSTRAT) ---
num_reefs = 150
reef_lats = np.random.uniform(-24, -9, num_reefs) # De Brisbane à la PNG
# On place les récifs autour de la courbe idéale
base_lons = get_reef_curve(reef_lats)
reef_lons = base_lons + np.random.uniform(-0.4, 0.6, num_reefs) # Largeur du récif

# --- 3. ÉTAT DE SANTÉ INITIAL (SCÉNARIO CATASTROPHE) ---
reef_health = np.zeros(num_reefs)

# Zone PNG (Le Réservoir) : SAIN (100%)
idx_png = np.where(reef_lats > -11.7)[0]
reef_health[idx_png] = 1.0 

# Tout le reste (Nord, Centre, Sud) : DÉVASTÉ (0% - Rouge)
# On part du principe qu'on veut montrer la régénération totale
idx_gbr = np.where(reef_lats <= -11.7)[0]
reef_health[idx_gbr] = 0.0 

# --- 4. LES LARVES (AGENTS) ---
n_larvae = 100
# Initialisation dans le RÉSERVOIR (Cercle en haut)
larvae_lats = np.random.uniform(-9.5, -9, n_larvae)
larvae_lons = np.random.uniform(143.5, 145.5, n_larvae)

# --- 5. VISUEL ---
fig = plt.figure(figsize=(11, 13))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# A. CARTE DÉZOOMÉE (Vue Globale)
ax.set_extent([130, 170, -1, -34], crs=ccrs.PlateCarree()) # Jusqu'à -7 (PNG)


# Habillage "Dark Mode" Pro
ax.add_feature(cfeature.OCEAN, facecolor='#0b1521')
ax.add_feature(cfeature.LAND, facecolor='#2b2b2b', edgecolor='black')
ax.coastlines(resolution='50m', color='#555555', linewidth=0.8)

# B. DESSIN DU RÉSERVOIR (ZONE ARRONDIE)
# Un cercle vert transparent pour symboliser la "Source de Vie"
reservoir_circle = mpatches.Circle((144.5, -1), radius=1.3, color='none', 
                                   transform=ccrs.PlateCarree(), zorder=0)
ax.add_patch(reservoir_circle)

# Contour du cercle
reservoir_outline = mpatches.Circle((145,-10), radius=1.6, fill=False, edgecolor='cyan', 
                                    linestyle='--', linewidth=1.5, transform=ccrs.PlateCarree(), zorder=0)
ax.add_patch(reservoir_outline)

ax.text(141, -8, 'RÉSERVOIR LARVAIRE\n(Papouasie / Torres)', color='cyan', fontsize=10, fontweight='bold', transform=ccrs.PlateCarree())
ax.text(150, -16, 'COURANT EAC\n(Flux Régénérateur)', color='white', fontsize=10, fontweight='bold', transform=ccrs.PlateCarree())

# C. LES RÉCIFS (Fixes)
cmap = plt.cm.RdYlGn # Rouge -> Vert
scat_reefs = ax.scatter(reef_lons, reef_lats, c=reef_health, cmap=cmap, vmin=0, vmax=1,
                        s=20, marker='2', alpha=0.8, edgecolors='none', transform=ccrs.PlateCarree(), zorder=5)

# D. LES LARVES (Flux)
scat_larvae = ax.scatter(larvae_lons, larvae_lats, s=5, c='cyan', 
                         edgecolors='black', linewidth=0.2, alpha=0.5, transform=ccrs.PlateCarree(), zorder=10)

# --- 6. UPDATE ---
def update(frame):
    global larvae_lons, larvae_lats, reef_health
    
    # 1. DÉPLACEMENT (GUIDAGE PAR LE COURANT)
    # On calcule la "route idéale" (le centre du récif) pour chaque larve
    ideal_route_lon = get_reef_curve(larvae_lats)
    
    # Force Latérale : Si la larve s'éloigne trop de la route, le courant la ramène
    drift_correction = (ideal_route_lon - larvae_lons) * 1
    
    # Vitesse Sud : Accélère un peu en descendant
    speed_south = -1.2 - (np.random.rand(n_larvae) * 0.5)
    
    # Application du mouvement
    larvae_lons += drift_correction + np.random.normal(0, 0.3, n_larvae) +1# + un peu de chaos
    larvae_lats += speed_south * 0.5 # Pas de temps
    
    # 2. RESPAWN (BOUCLE INFINIE)
    # Si elles sortent en bas (-30), elles reviennent dans le RÉSERVOIR (-9)
    mask_out = larvae_lats < -40
    
    if np.sum(mask_out) > 0:
        # Respawn dans le cercle du haut
        larvae_lats[mask_out] = np.random.uniform(-10, -8, np.sum(mask_out))
        larvae_lons[mask_out] = np.random.uniform(143.5, 145.5, np.sum(mask_out))
    
    # 3. RÉGÉNÉRATION (INTERACTION)
    # Les récifs rouges (morts) deviennent verts s'ils sont touchés par le flux
    
    # On regarde seulement les larves qui sont en dessous du réservoir
    active_larvae_idx = np.where(larvae_lats < -10.5)[0]
    
    if len(active_larvae_idx) > 0:
        # Pour optimiser : on ne cherche pas la distance exacte larve par larve (trop lent)
        # On regarde où est le "front" de la vague de larves
        # Tous les récifs qui sont à la même latitude qu'un groupe de larves guérissent
        
        min_lat_larvae = np.min(larvae_lats)
        max_lat_larvae = np.max(larvae_lats[active_larvae_idx]) if len(active_larvae_idx) > 0 else -10
        
        # On cible les récifs dans cette bande de latitude
        # Et qui sont proches de la longitude du flux
        # C'est une approximation pour l'animation visuelle rapide
        
        # On triche un peu pour le spectacle : on soigne les récifs "proches" du flux
        # On prend 30 récifs au hasard parmi ceux qui sont malades et qui sont "sous" le flux
        sick_reefs = np.where(reef_health < 1.0)[0]
        
        if len(sick_reefs) > 0:
            # On filtre ceux qui sont géographiquement proches des larves actuelles
            # (Latitude correspondante)
            # On prend un échantillon aléatoire pour aller vite
            sample_sick = np.random.choice(sick_reefs, size=min(len(sick_reefs), 50))
            
            # Vérification simple de latitude
            for r_idx in sample_sick:
                # Si le récif est à une latitude où il y a des larves...
                if min_lat_larvae < reef_lats[r_idx] < max_lat_larvae:
                    reef_health[r_idx] += 0.1 # GUÉRISON RAPIDE
    
    # Clamp health
    reef_health = np.clip(reef_health, 0, 1)

    # Update graphs
    scat_larvae.set_offsets(np.column_stack((larvae_lons, larvae_lats)))
    scat_reefs.set_array(reef_health)
    
    return scat_larvae, scat_reefs

plt.arrow(146,-12,7,-10,length_includes_head=10,head_width=0.5,head_length=0.7,color='white',transform=ccrs.PlateCarree())

print("Simulation du flux régénérateur Papouasie -> GBR...")
anim = FuncAnimation(fig, update, frames=250, interval=30, blit=False)
anim.save('EAC_help_regen.gif', writer='pillow', fps=20, dpi=100)

plt.show()

/opt/miniconda3/envs/gbr_analysis/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:1677: UserWarning: You passed a edgecolor/edgecolors ('none') for an unfilled marker ('2').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  result = super().scatter(*args, **kwargs)


Simulation du flux régénérateur Papouasie -> GBR...
